In [ ]:
# Import MySQL Connector/Python 
from mysql.connector.pooling import  MySQLConnectionPool
from mysql.connector import Error

# Set configuration
# replace host by "127.0.0.1" if working with database on localhost
# replace password and user fields with your credentials

dbconfig = {
    "user" : "capstone",
    "password" : "password",
    "host" : "192.168.56.2",
    "database" : "LittleLemonDB"
}

pool_name = "pool_a"
pool_size = 5

In [ ]:
# Almost every action on the database can fail (for example due to network failure)
# Therefore, it's good to handle exceptions on every connection and cursor method call
# request. But the pattern will always be the same:
# try:
#    print(<Doing something>)
#    <cursor/connection method call>
#    print(<Done something>)
# except connector.Error as err:
#    print(<Error doing something>)
#    print("Error code:", err.errno)
#    print("Error message:", err.msg)
#
# In order to catch all these exceptions and not to repeat the same code multiple times,
# I'll write a wrapper for all the calls inside this construction with a single line of 
# code afterwards. I would prefer decorator, but decorators do not seem to work without 
# def's As a bonus, it will enforce me to write suitable descriptions in the 'print'
# statements as required by the assignement guidelines.

def log_and_catch(action_str, func, *args, **kwargs):
    try:
        print(action_str)
        result = func(*args, **kwargs)
        print("Success")
        return result
    except Error as err:
        print("Error", action_str)
        print("Error code:", err.errno)
        print("Error message:", err.msg, "\n")
        return None

In [ ]:
# Create the pool. log_and_catch defined above provides try .. except error handling

pool = log_and_catch(
    "Creating the connection pool %s for the database %s" % (pool_name, dbconfig["database"]),
    MySQLConnectionPool,
    pool_name=pool_name,
    pool_size=pool_size,
    **dbconfig)

In [ ]:
connection = log_and_catch(
    "Obtaining conection from the pool",
    pool.get_connection)

In [ ]:
cursor = log_and_catch(
    "Allocating cursor",
    connection.cursor,
    buffered=True)

In [ ]:
show_tables_query = "SHOW tables" 

log_and_catch(
    "Querying tables",
    cursor.execute,
    show_tables_query)

results = log_and_catch(
    "Fetching data",
    cursor.fetchall)

for r in results:
    print(r)

In [ ]:
promo_query = """
SELECT
FirstName,
LastName,
Email
FROM
Customers JOIN Orders ON Orders.CustomerID = Customers.ID
WHERE TotalCost > 60
"""

log_and_catch(
    "Querying Users Promo Details",
    cursor.execute,
    promo_query)

results = log_and_catch(
    "Fetching data",
    cursor.fetchall)

for r in results:
    print(r)